In [162]:
from jsonobject import *
import numpy as np 
import random
import math
import networkx as nx
from networkx.readwrite import json_graph
import matplotlib as mpl
import matplotlib.pyplot as plt
from enum import Enum

In [163]:
random.seed(2021)

# Classes

In [164]:
# Convolution
class Convolution(JsonObject):
    kernel = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    nb_filter = IntegerProperty()
    fct_activation = StringProperty()

In [165]:
# Input 
class InputLayer(JsonObject):
    shape = ListProperty(int)

In [166]:
# Pooling Avg/Max
class Pooling(JsonObject):
    op = StringProperty()
    kernel = IntegerProperty(default=2)
    padding = StringProperty(default="'valid'")
    stride = IntegerProperty(default=None)

In [167]:
# Class Flatten
class Flatten(JsonObject):
    pass

In [168]:
# Dense --> Fully connected layer
class Dense(JsonObject):
    nb_neurones =  IntegerProperty()
    fct_activation = StringProperty()

In [169]:
# identity block 
class IdBlock(JsonObject):
    kernel = IntegerProperty()
    padding = StringProperty()
    nb_filter = IntegerProperty()
    stride = 1
    fct_activation = StringProperty()

In [170]:
# convolution block
class ConvBlock(JsonObject):
    kernel = IntegerProperty()
    nb_filter = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    fct_activation = StringProperty()

In [171]:
# Enum type archi
class TypeArchi(Enum):
    ALL = 0
    LENET = 1
    RESNET = 2
    DENSENET = 3

In [172]:
# configuration Archi
class ConfArchi(JsonObject):
    type_archi = IntegerProperty()
    epsilon = FloatProperty()
    dropout_rate = FloatProperty()
    compress_factor = FloatProperty()

In [173]:
# DenseNet Convolution Block 
class DenseNetBlock(JsonObject):
    kernel = IntegerProperty()
    nb_filter = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    nb_layer = IntegerProperty()
    fct_activation = StringProperty()
    op = StringProperty()

In [174]:
# Global pooling 
class GlobalPooling(JsonObject):
    op = StringProperty()

# Value

In [175]:
# Table of hyperparameter value
kernel_value = [2, 3, 4, 5, 6, 7]
stride_value = [1, 2, 3, 4, 5, 6, 7]
padding_value = ["'valid'", "'same'"]
fct_activation_value = ["tanh", "relu", "selu"]

epsilon_value = [None, 1.001e-5, 0.001, 1.1e-5, 1.1e-7]
dropout_value = [None, .1, .4, .5, .8, .01, .001]

nb_layer_value = [1,2]

nb_class = 10 # nb_class
op_value = ['avg',"max"]

In [176]:
epsilon_value

[None, 1.001e-05, 0.001, 1.1e-05, 1.1e-07]

# Functions

In [177]:
def stop(size_archi, x):
    prob = (math.log(2*size_archi)*x)-(2*x)
    if ( prob < random.randrange(101)):
        return True
    else : return False

In [178]:
def create_json_file(archi, file_name):
    directory = 'architecture_json/'
    
    # reset file
    archi_file = open(directory+file_name, "w")
    archi_file.close()
    
    # create file
    
    # open file
    archi_file = open(directory+file_name, "a")
    
    archi_file.write("""[
    """)
    
    archi_size = len(archi)
    i = 0
    for l in archi:

        str_layer = """\t{
            'class':'"""
        str_layer += l.__class__.__name__
        str_layer +="""',\n\t\t\t'parameters':"""
        str_layer += str(l.to_json())
        str_layer += """\n\t\t}"""
        if(i < archi_size-1):
            str_layer += ""","""
            i+=1
        str_layer = str_layer.replace("'","\"")
        
        archi_file.write(str_layer)
        
    archi_file.write("""\n]""")
    archi_file.close()

In [179]:
# compute the output 
# valid vs same
def calcul_output(input_size, l):
    output_size = 0
    if(l.padding == "valid"):
        kernel = l.kernel
        stride = l.stride
        while(input_size>=kernel):
            input_size -= stride
            output_size += 1   
    else:
        stride = l.stride
        if(input_size%stride == 0):
            output_size = int(input_size/stride)
        else:
            output_size = int(input_size/stride)+1
    return output_size

In [180]:
# add layer to architecture 

def addLayer(archi, layer):
    input_size = archi[0].shape[0]
    feature_extra = archi[1:]
    
    # add layer if the architecture is empty
    if(feature_extra == []):
        archi.append(layer)
        return 1, archi, calcul_output(input_size,layer)
    
    else :
        # compute size of the output of the last layer
        for l in feature_extra: 
            output_size = calcul_output(input_size, l)
            input_size = output_size
        
        # if we couldn't reduce more
        if(input_size == 1):
            return 0, archi, input_size
        
        # if the output size got more than 1 we can add new layer
        elif(output_size > 1): 
            output_size = calcul_output(input_size, layer)
            
            # if output size got negate is that the layer we want to add is wrong
            if(output_size < 1 ):
                return -1, archi, input_size
            # if output size is bigger than 0 we can add new layer and continue
            elif(output_size > 0):
                archi.append(layer)
                return 1, archi, output_size
            # this should not append
            else:
                return "Somethink wrong"
    # this should not append
    return "Error"

# Config Architecture

In [181]:
esp = epsilon_value[random.randrange(5)]
rate = dropout_value[random.randrange(7)]

# we can force the generator to generate one type of architecture in particular
type_archi = TypeArchi.DENSENET

ca = ConfArchi(type_archi=type_archi.value, epsilon=esp, dropout_rate=rate, compress_factor=0.5)

In [182]:

full_extraction_feature = [] # full feature extraction
wp_extraction_feature = [] # feature extraction without pooling

# all layer
if (type_archi == TypeArchi.ALL):
    full_extraction_feature= [Convolution, Pooling, IdBlock, ConvBlock, DenseNetBlock] 
    wp_extraction_feature= [Convolution, IdBlock, ConvBlock, DenseNetBlock] 
# if Lenet
elif (type_archi == TypeArchi.LENET):
    full_extraction_feature = [Convolution, Pooling]
    wp_extraction_feature = [Convolution]
# if resnet
elif (type_archi == TypeArchi.RESNET):
    full_extraction_feature= [Convolution, Pooling, IdBlock, ConvBlock]
    wp_extraction_feature= [Convolution, IdBlock, ConvBlock]
# if denset
else: 
    full_extraction_feature= [Convolution, Pooling, DenseNetBlock]
    wp_extraction_feature= [Convolution, DenseNetBlock]

classification = [Flatten, GlobalPooling]

# Creation d'architecture

In [190]:
architecture = list()

nb_filter_value = 1 # init nb feature map
output_size = 28

# add input Layer
architecture.append(InputLayer(shape=[28,28,1]))

# add extraction feature (succession of Pooling/convolution)
# Pooling can't be follow by a Pooling
pooling = True
code = 1 # we can add new layer
j = 6
while((output_size == 32) | ((stop(len(architecture),7)) & (code == 1))):
    # not get 2 pooling in a row
    if(pooling):
        layer = wp_extraction_feature[random.randrange(len(wp_extraction_feature))]
        pooling = False
    else:
        layer = full_extraction_feature[random.randrange(len(full_extraction_feature))]
        if(layer == Pooling):
            pooling = True
            
    # incrementation of number of filter
    if(layer == Convolution or layer == ConvBlock):
        nb_filter_value = nb_filter_value*j
        j=2

        
        
    # kernel >= stride
    kernel=kernel_value[random.randrange(6)]
    stride_value_filtered = [value for value in stride_value if value <= kernel]
    stride = stride_value_filtered[random.randrange(len(stride_value_filtered))]
    
    # if layer is Convolution
    if(layer == Convolution):
        add_layer = Convolution(
            kernel=kernel, 
            padding=padding_value[random.randrange(2)], 
            stride=stride, 
            nb_filter= nb_filter_value,
            fct_activation=fct_activation_value[random.randrange(3)]
        )
    # if is pooling
    elif(layer == Pooling):
        add_layer = Pooling(
            op = op_value[random.randrange(2)],
            kernel=kernel, 
            padding=padding_value[random.randrange(2)], 
            stride=stride
        )
    # if is identity block
    elif(layer == IdBlock):
        add_layer = IdBlock(
            kernel=kernel, 
            padding='same', 
            nb_filter=nb_filter_value, 
            fct_activation=fct_activation_value[random.randrange(3)])
        # if is convolution block
    elif (layer == ConvBlock):
        add_layer = ConvBlock(
            kernel=kernel, 
            nb_filter=nb_filter_value, 
            padding='same', 
            stride=stride,
            fct_activation=fct_activation_value[random.randrange(3)])
    elif (layer == DenseNetBlock):
        add_layer = DenseNetBlock( 
            kernel = kernel, 
            nb_filter = nb_filter_value, 
            padding='same', 
            nb_layer = nb_layer_value[random.randrange(2)],
            stride = stride,
            fct_activation=fct_activation_value[random.randrange(3)],
            op = op_value[random.randrange(2)]
        )
    else: 
        print("error")
    
    code, architecture, output_size = addLayer(architecture, add_layer)

try:
    nb_filter_value = architecture[len(architecture)-1]['nb_filter']
except:
    nb_filter_value = int(nb_filter_value/2)

print("output size : " + str(output_size))
print("nb_filter : " + str(nb_filter_value))
print("code: " + str(code))


clf_layer = classification[random.randrange(2)]
print(clf_layer)
if clf_layer == Flatten:
    # add flatten layer
    architecture.append(Flatten())


    # compute parameters
    param = output_size*output_size*(nb_filter_value)
    print('param : ' + str(param))


    ## init values
    nb=0
    if (param >= 1000):
        nb_neurones = int(param*20/100)
        print("=====")
        print("iteration" + str(nb))
        print("nb_neurone : " + str(nb_neurones))
        dense = Dense(
            nb_neurones = nb_neurones,
            fct_activation = fct_activation_value[random.randrange(3)]
        )
        architecture.append(dense)
        nb+=1
        nb_neurones = int(nb_neurones*20/100)
        
    else:
        pourcent = random.uniform(10,90)
        nb_neurones = int(param*pourcent/100)

    while(nb_neurones >= 1000):
        print("=====")
        print("iteration" + str(nb))
        print("nb_neurone : " + str(nb_neurones))
        dense = Dense(
            nb_neurones = nb_neurones,
            fct_activation = fct_activation_value[random.randrange(3)]
        )
        architecture.append(dense)
        nb+=1
        nb_neurones = int(nb_neurones*20/100)
        
        
    ## add dense Layers
    while(nb_neurones > nb_class*10):
        print("=====")
        print("iteration" + str(nb))
        print("nb_neurone : " + str(nb_neurones))

        dense = Dense(
            nb_neurones = nb_neurones,
            fct_activation = fct_activation_value[random.randrange(3)]
        )
        architecture.append(dense)


        #incrementation
        pourcent = random.uniform(10,90)
        nb+=1
        nb_neurones = int(nb_neurones*pourcent/100)
    
elif (clf_layer == GlobalPooling):
    architecture.append(GlobalPooling(op=op_value[random.randrange(2)]))
else: print(clf_layer == GlobalPooling)
    

last_dense = Dense(
    nb_neurones = nb_class,
    fct_activation="softmax"           
)

architecture.append(last_dense)

# ajout config
architecture.append(ca)

# print architecture
print(architecture)

# create file
file_name = "archi_V3_test.json"
print(file_name)
print("")
print("")
create_json_file(architecture, file_name)
        

output size : 7
nb_filter : 6
code: 1
<class '__main__.GlobalPooling'>
[InputLayer(shape=[28, 28, 1]), Convolution(fct_activation='relu', kernel=3, nb_filter=6, padding='same', stride=1), DenseNetBlock(fct_activation='relu', kernel=3, nb_filter=6, nb_layer=2, op='avg', padding='same', stride=2), DenseNetBlock(fct_activation='selu', kernel=3, nb_filter=6, nb_layer=2, op='max', padding='same', stride=1), DenseNetBlock(fct_activation='tanh', kernel=5, nb_filter=6, nb_layer=2, op='avg', padding='same', stride=2), GlobalPooling(op='max'), Dense(fct_activation='softmax', nb_neurones=10), ConfArchi(compress_factor=0.5, dropout_rate=0.001, epsilon=1.1e-05, type_archi=3)]
archi_V3_test.json


